# Import libraries

In [77]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import Datasets

In [78]:
train = pd.read_excel('C:\\Users\\erkai\\OneDrive\\Documents\\Data_Train.xlsx')

In [79]:
test = pd.read_excel('C:\\Users\\erkai\\OneDrive\\Documents\\Data_Test.xlsx')

In [80]:
train.shape , test.shape

((12690, 9), (4231, 8))

In [81]:
train.duplicated().sum()

25

In [82]:
test.duplicated().sum()

1

In [83]:
train.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [84]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12690 entries, 0 to 12689
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TITLE          12690 non-null  object
 1   RESTAURANT_ID  12690 non-null  int64 
 2   CUISINES       12690 non-null  object
 3   TIME           12690 non-null  object
 4   CITY           12578 non-null  object
 5   LOCALITY       12592 non-null  object
 6   RATING         12688 non-null  object
 7   VOTES          11486 non-null  object
 8   COST           12690 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 892.4+ KB


In [85]:
for i in train.columns:
    print("unique values in", i, train[i].nunique())


unique values in TITLE 113
unique values in RESTAURANT_ID 11892
unique values in CUISINES 4155
unique values in TIME 2689
unique values in CITY 359
unique values in LOCALITY 1416
unique values in RATING 32
unique values in VOTES 1847
unique values in COST 86


# Data Preprocessing

In [86]:
df = train.append(test,ignore_index=True)

In [87]:
df = df[['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY', 'RATING', 'VOTES', 'COST']]

In [88]:
def extract_closed(time):
    a = re.findall('closed\(.*?\)', time)
    if a !=[]:
        return a[0]
    else:
        return 'NA'
    
    df['CLOSED'] = df['TIME'].apply(extract_closed)

In [89]:
df['TIME'] = df['TIME'].str.replace(r'closed\(*?\)', '')

In [90]:
df['RATING'] = df['RATING'].str.replace('NEW', '1')
df['RATING'] = df['RATING'].str.replace('-', '1').astype(float)

In [91]:
df['VOTES'] = df['VOTES'].str.replace('votes', '').astype(float)

In [92]:
df['CITY'].fillna('Missing', inplace=True)
df['LOCALITY'].fillna('Missing', inplace=True)
df['RATING'].fillna(3.8, inplace=True)
df['RATING'].fillna(0.0, inplace=True)

In [93]:
df['COST'] =df['COST'].astype(float)

In [94]:
df.head(2)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49.0,1200.0
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30.0,1500.0


In [95]:
df['TITLE'].nunique(),df['CUISINES'].nunique()

(123, 5183)

In [96]:
calc_mean = df.groupby(['CITY'], axis=0).agg({'RATING':'mean'}).reset_index()
calc_mean.columns = ['CITY', 'CITY_MEAN_RATING']
df = df.merge(calc_mean, on=['CITY'], how='left')

calc_mean = df.groupby(['LOCALITY'], axis=0).agg({'RATING': 'mean'}).reset_index()
calc_mean.columns = ['LOCALITY', 'LOCALITY_MEAN_RATING']
df = df.merge(calc_mean, on=['LOCALITY'], how='left')

In [97]:
df.head(2)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST,CITY_MEAN_RATING,LOCALITY_MEAN_RATING
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49.0,1200.0,3.376271,3.388889
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30.0,1500.0,3.584588,3.472222


In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [99]:
corpus = ['TITLE','RESTAURANT_ID','CUISINES', 'TIME', 'CITY', 'LOCALITY', 'RATING', 'VOTES', 'COST', 'CITY_MEAN_RATING', 'LOCALITY_MEAN_RATING']

In [100]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [101]:
df.drop(['TITLE', 'CUISINES', 'CITY', 'LOCALITY', 'TIME'], axis=1, inplace=True)

In [102]:
df = pd.get_dummies(df, columns=['CLOSED'], drop_first=True)

KeyError: "None of [Index(['CLOSED'], dtype='object')] are in the [columns]"

In [ ]:
df.shape

In [ ]:
train_df = df[df['COST'].isnull()!=True]
test_df = df[df['COST'].isnull()==True]
test_df.drop('COST', axis=1, inplace=True)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df['COST'] = np.log1p(train_df['COST'])

# Train Test Split

In [ ]:
X = train_df.drop(labels=['COST'], axis=1)
y = train_df['COST'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

# Build The Model

In [ ]:
from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'l2_root',
         'learning_rate': 0.05, 
         'num_iterations': 350,
         'num_leaves': 31,
         'max_depth': -1,
         'min_data_in_leaf': 15,
         'bagging_fraction': 0.85,
         'bagging_freq': 1,
         'feature_fraction': 0.55
         }

lgbm = lgb.train(params=param,
                 verbose_eval=50,
                 train_set=train_data,
                 valid_sets=[test_data])

y_pred_lgbm = lgbm.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_lgbm))))

In [ ]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor(base_estimator=None, n_estimators=30, max_samples=0.9, max_features=1.0, bootstrap=True, 
                      bootstrap_features=True, oob_score=True, warm_start=False, n_jobs=1, random_state=42, verbose=1)
br.fit(X_train, y_train)
y_pred_br = br.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_br))))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=40, criterion='mse', max_depth=None, min_samples_split=4, min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                           min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, 
                           random_state=42, verbose=1, warm_start=False)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_rf))))

In [ ]:
y_pred = y_pred_lgbm*0.70 + y_pred_br*0.15 +  y_pred_rf*0.15
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred))))